# Classical Model for CO₂ + NH₃ → NH₂COOH
This notebook demonstrates classical quantum chemistry analysis of molecular geometries, vibrational modes, and reaction energetics.

We use dummy Gaussian outputs parsed with a mock cclib interface to simulate the workflow.

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:

class DummyCclibData:
    def __init__(self, vibfreqs, scfenergies, atomcoords, natom):
        self.vibfreqs = vibfreqs
        self.scfenergies = scfenergies
        self.atomcoords = atomcoords
        self.natom = natom

class DummyCclibIO:
    def ccread(self, logfile):
        if "CO2_freq.log" in logfile:
            return DummyCclibData(
                vibfreqs=[667.4, 1388.3, 2349.1],
                scfenergies=[-188.5879799],
                atomcoords=[[[0.0, 0.0, 0.0], [0.0, 0.0, 1.16], [0.0, 0.0, -1.16]]],
                natom=3
            )
        elif "NH3_freq.log" in logfile:
            return DummyCclibData(
                vibfreqs=[950.0, 1627.0, 3444.0, 3444.0],
                scfenergies=[-56.5879799],
                atomcoords=[[[0.0, 0.0, 0.0], [0.0, 0.9366, 0.381], [0.8115, -0.4683, 0.381], [-0.8115, -0.4683, 0.381]]],
                natom=4
            )
        elif "NH2COOH_freq.log" in logfile:
            return DummyCclibData(
                vibfreqs=[500.0, 700.0, 900.0, 1200.0, 1400.0, 1600.0, 1700.0, 2900.0, 3000.0, 3200.0, 3300.0],
                scfenergies=[-245.1879799],
                atomcoords=[[[0.0, 0.0, 0.0], [-1.2, 0.0, 0.0], [1.0, 0.7, 0.0], [0.5, -1.2, 0.0], [1.5, -1.2, 0.0], [0.0, -2.0, 0.0], [1.8, 0.4, 0.0]]],
                natom=7
            )
        else:
            raise Exception(f"Log file not found: {logfile}")

io = DummyCclibIO()


In [ ]:

def parse_gaussian_freq(logfile):
    data = io.ccread(logfile)
    frequencies = data.vibfreqs if hasattr(data, 'vibfreqs') else []
    final_energy = data.scfenergies[-1] if hasattr(data, 'scfenergies') else None
    final_geometry = data.atomcoords[-1] if hasattr(data, 'atomcoords') else None
    return {
        'frequencies': frequencies,
        'energy': final_energy,
        'geometry': final_geometry,
        'n_atoms': data.natom if hasattr(data, 'natom') else 0
    }

def calculate_reaction_energy(e_co2, e_nh3, e_product):
    delta_e = e_product - (e_co2 + e_nh3)
    delta_e_kcal = delta_e * 627.509
    return delta_e, delta_e_kcal

def calculate_zpe(frequencies):
    return 0.5 * sum(freq * 4.5563e-6 for freq in frequencies)


In [ ]:

co2_data = parse_gaussian_freq("CO2_freq.log")
nh3_data = parse_gaussian_freq("NH3_freq.log")
product_data = parse_gaussian_freq("NH2COOH_freq.log")

e_co2 = co2_data['energy']
e_nh3 = nh3_data['energy']
e_product = product_data['energy']

delta_e_hartree, delta_e_kcal = calculate_reaction_energy(e_co2, e_nh3, e_product)

zpe_co2 = calculate_zpe(co2_data['frequencies'])
zpe_nh3 = calculate_zpe(nh3_data['frequencies'])
zpe_product = calculate_zpe(product_data['frequencies'])
delta_e_zpe = (e_product + zpe_product) - (e_co2 + zpe_co2 + e_nh3 + zpe_nh3)
delta_e_zpe_kcal = delta_e_zpe * 627.509

results = {
    "Electronic Reaction Energy (Hartree)": delta_e_hartree,
    "Electronic Reaction Energy (kcal/mol)": delta_e_kcal,
    "Reaction Energy + ZPE (Hartree)": delta_e_zpe,
    "Reaction Energy + ZPE (kcal/mol)": delta_e_zpe_kcal
}

df = pd.DataFrame([results])
df.to_csv("classical_model_with_zpe.csv", index=False)
df


In [ ]:

molecules = ['CO2', 'NH3', 'NH2COOH']
all_frequencies = [co2_data['frequencies'], nh3_data['frequencies'], product_data['frequencies']]

plt.figure(figsize=(10, 6))
for i, freqs in enumerate(all_frequencies):
    plt.bar(np.arange(len(freqs)) + i * 0.25, freqs, width=0.25, label=molecules[i])
plt.xlabel('Vibrational Mode Index')
plt.ylabel('Frequency (cm⁻¹)')
plt.title('Simulated Vibrational Frequencies of CO2, NH3, and NH2COOH')
plt.legend()
plt.tight_layout()
plt.savefig('vibrational_frequencies_with_zpe.png')
plt.show()
